<a href="https://colab.research.google.com/github/mnocerino23/NBA-Player-Classifier/blob/main/NeuralNetwork_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building a multi-class classifier with Neural Networks

In [73]:
import pandas as pd
import sklearn
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
complete = pd.read_csv('/content/drive/MyDrive/Data_Science_Projects/NBA/complete22-23.csv')

In [75]:
complete.head(4)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST_TOV,OWS_div_USG,TS_times_USG,bigmanD,foul_rate,three_rate,PTs_per_36,PF_per_dplay,FG_TOV,two_way
0,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,0.485,...,0.818182,0.041237,10.7476,0.251208,0.091787,21.505376,16.000000,1.727273,3.727273,38.690909
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,...,1.210526,0.089041,8.2344,0.314815,0.085185,0.000000,11.466667,1.150000,1.947368,87.500000
2,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,0.540,...,1.280000,0.142857,14.9184,0.251445,0.080925,1.324503,21.225434,1.400000,3.200000,101.840000
3,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,0.427,...,1.571429,0.056962,8.8638,0.092683,0.082927,37.500000,13.873171,2.833333,6.000000,6.440678


In [76]:
#we want to make sure each player has only one position to limit the amount of categories for Pos.
#although we don't need to one hot encode for our tree based model, limiting all 
#entries to just one position is still important

for index, rows in complete.iterrows():
  #if the position has length greater than two (meaning it isn't C, PG, SG, SF, or PF), we shorten it to the first position
  if len(complete.at[index, 'Pos']) > 2:
    complete.at[index, 'Pos'] = complete.at[index, 'Pos'][:2]

# Players with salaries over 25 million will be considered max level players while those under with be considered "Not max"

In [77]:
#use a lambda function to assign classes
complete['Class'] = ''

for index, rows in complete.iterrows():
  salary = complete.at[index,'Salary']
  if salary <= 10:
    complete.at[index, 'Class'] = 'Lower-Tier'
  elif salary > 10 and salary <=25:
    complete.at[index, 'Class'] = 'Mid-Tier'
  else:
    complete.at[index, 'Class'] = 'Star'

In [78]:
#the count of each class in the dataset before splitting
complete['Class'].value_counts()

Lower-Tier    312
Mid-Tier       86
Star           52
Name: Class, dtype: int64

Before splitting into train-test, we need to drop outliers that have been identified such as Kyle Lowry, John Wall, Gordon Hayward, and Ben Simmons. These are players who have all severly fallen off after receiving max contracts and should not appear in the training set.

In [79]:
#need to remove john wall, Gordon Hayward, Ben Simmons from the training set

In [80]:
need_to_remove = ['John Wall', 'Gordon Hayward', 'Ben Simmons']
for player in need_to_remove:
  complete = complete.drop(complete[complete['Player'] == player].index)

In [81]:
print(list(complete['Player']))

['Precious Achiuwa', 'Steven Adams', 'Bam Adebayo', 'Ochai Agbaji', 'Santi Aldama', 'Nickeil Alexander-Walker', 'Grayson Allen', 'Jarrett Allen', 'Jose Alvarado', 'Kyle Anderson', 'Giannis Antetokounmpo', 'Thanasis Antetokounmpo', 'Cole Anthony', 'OG Anunoby', 'Ryan Arcidiacono', 'Deni Avdija', 'Deandre Ayton', 'Udoka Azubuike', 'Marvin Bagley III', 'Patrick Baldwin Jr.', 'LaMelo Ball', 'Mo Bamba', 'Paolo Banchero', 'Desmond Bane', 'Dalano Banton', 'Dominick Barlow', 'Harrison Barnes', 'Scottie Barnes', 'RJ Barrett', 'Will Barton', 'Charles Bassey', 'Keita Bates-Diop', 'Nicolas Batum', 'Darius Bazley', 'Bradley Beal', 'Malik Beasley', 'MarJon Beauchamp', 'Dāvis Bertāns', 'Patrick Beverley', 'Saddiq Bey', 'Khem Birch', 'Goga Bitadze', 'Bismack Biyombo', 'Bogdan Bogdanović', 'Bojan Bogdanović', 'Bol Bol', 'Devin Booker', 'Brandon Boston Jr.', 'Chris Boucher', 'James Bouknight', 'Malaki Branham', 'Christian Braun', 'Mikal Bridges', 'Oshae Brissett', 'Malcolm Brogdon', 'Dillon Brooks', 'Br

In [82]:
from sklearn.model_selection import train_test_split, cross_val_score

In [83]:
#split into input features and target variable class
X = complete.drop(columns = ['Salary', 'Class'], axis = 1)
y = complete['Class']

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 5, test_size=0.2, shuffle = True)

In [85]:
#X_train.loc[X_train['Class'] == 1]['Player']

In [86]:
#drop the names column before proceeding
names_train = list(X_train['Player'])
X_train.drop(columns = ['Player', 'Pos', 'Tm'], axis = 1, inplace = True)
names_test = list(X_test['Player'])
X_test.drop(columns = ['Player', 'Pos', 'Tm'], axis = 1, inplace = True)

In [87]:
print('Train:')
print('train_features dimensions: ', X_train.shape)
print('train_target dimensions: ', y_train.shape, '\n')


print('Test:')
print('test_features dimensions: ', X_test.shape)
print('test_target dimensions: ', y_test.shape)

Train:
train_features dimensions:  (357, 59)
train_target dimensions:  (357,) 

Test:
test_features dimensions:  (90, 59)
test_target dimensions:  (90,)


In [88]:
#use keras within tensorflow since I have used this in the past
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(5)

nn = tf.keras.models.Sequential()
nn.add(tf.keras.Input(shape=(59,)))
nn.add(tf.keras.layers.Dense(15, activation='relu'))
nn.add(tf.keras.layers.Dense(30, activation='relu'))
nn.add(tf.keras.layers.Dense(30, activation='relu'))
nn.add(tf.keras.layers.Dense(15, activation='relu'))
nn.add(tf.keras.layers.Dense(3, activation='softmax'))

nn.compile(
   loss=tf.keras.losses.CategoricalCrossentropy(),
   optimizer=tf.keras.optimizers.Adam(),
   metrics="accuracy")

history = nn.fit(X_train, y_train, validation_data=(X_test,y_test))

ValueError: ignored